In [44]:
# Starr Report copied from 
# https://www.washingtonpost.com/wp-srv/politics/special/clinton/icreport/srprintable.htm
!ls

Untitled.ipynb     mueller-report.pdf mueller_report.txt starr_report.txt


In [36]:
import pdfminer
import re
import scattertext as st
import pandas as pd

In [10]:
from pdfminer import pdfparser

In [15]:
parser = pdfparser.PDFParser(open('mueller-report.pdf', 'rb'))


In [24]:
#From http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Using%20Python%20to%20Convert%20PDFs%20to%20Text%20Files.php

from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 
   

In [26]:
mueller_report_raw_text = convert('mueller-report.pdf')

In [28]:
open('mueller_report.txt', 'w').write(mueller_report_raw_text)

1326467

In [46]:
starr_report_raw = open('starr_report.txt').read()

In [62]:
def is_proper_paragraph(paragraph):
    return len(paragraph) > 200 and not re.match('^\d', paragraph) and '...................' not in paragraph

In [95]:
mueller_paragraphs = [paragraph.strip() for paragraph in mueller_report_raw_text.split('\n\n') 
                      if is_proper_paragraph(paragraph)]
mueller_df = pd.DataFrame({'text': mueller_paragraphs})
mueller_df['category'] = 'Mueller'

In [96]:
starr_report_paragraphs = [paragraph.strip() for paragraph in starr_report_raw.split('\n\n') 
                           if is_proper_paragraph(paragraph)]
starr_df = pd.DataFrame({'text': starr_report_paragraphs})
starr_df['category'] = 'Starr'

In [98]:
full_df = pd.concat([mueller_df, starr_df])

In [99]:
full_df['parse'] = full_df.text.apply(st.whitespace_nlp_with_sentences)

In [100]:
corpus = (st.CorpusFromParsedDocuments(full_df, 
                                       category_col='category', 
                                       parsed_col='parse').build().get_unigram_corpus())

In [115]:
term_scorer = st.CohensD(corpus).set_categories('Mueller', ['Starr'])
html = st.produce_scattertext_explorer(
    corpus,
    category='Mueller',
    not_category_name='Starr',
    term_scorer=st.RankDifference(),
    transform=st.Scalers.dense_rank,
    use_full_doc=True
)
file_name = 'mueller_vs_starr.html'
open(file_name, 'wb').write(html.encode('utf-8'))

2066351